# Part 1

## Dependencies



    nsepy: for dataset
    pandas: for data manupulation and analysis 
    numpy : for linear algebera 
    bokeh : for visualization


In [1]:
!pip install nsepy
from nsepy import get_history
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import brewer
from bokeh.palettes import Spectral11
import bokeh 
import itertools
from datetime import date
import random
from statsmodels.tsa.stattools import acf, pacf
from bokeh.plotting import figure, output_file, save



You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## getting the data set 
   **infy_data** contains the ohlv of infoysis stocks from 2015 to 2016
      
      
   **tcs_data** contains the ohlv of infoysis stocks from 2015 to 2016

In [2]:
infy_data=get_history('INFY', start=date(2015,1,1), end=date(2015,12,31))
tcs_data=get_history('TCS', start=date(2015,1,1), end=date(2015,12,31))

infy_data=infy_data[['Open', 'High', 'Low', 'Close','Volume']]
tcs_data=tcs_data[['Open', 'High', 'Low', 'Close','Volume']]


infy_data = infy_data.asfreq('D', method ='pad')        # pad-ffill : forward-fill
tcs_data = tcs_data.asfreq('D', method ='pad')


In [3]:
tcs_data.insert(0, 'Date',  pd.to_datetime(tcs_data.index,format='%Y-%m-%d') )
infy_data.insert(0, 'Date',  pd.to_datetime(infy_data.index,format='%Y-%m-%d') )

## About the moving_av function 

    parameter
    1.series
    2.window (list of integers which we comprises the size of window)

    return
    1.dataframe with the column of average window size of each integer in the window list 

In [4]:
def moving_av(series, window):
  MA_df=pd.DataFrame()
  if len(window)==1:
    MA_df['MA of {}'.format(window)]=series.resample('W').interpolate().rolling(window[0]).mean()
    
  else:
    for i in window:
       MA_df['MA of {} weeks'.format(i)]=series.resample('W').interpolate().rolling(i).mean()
  
  return MA_df
      

## variable information
1. **tcs_moving_av** dataframe contains the column of moving average of the window size of [4,16,28,40,52]
2. **infy_moving_av** dataframe contains the column of moving average of the window size of [4,16,28,40,52]

In [5]:
tcs_moving_av=moving_av(tcs_data.Close, [4,16,28,40,52])
infy_moving_av=moving_av(infy_data.Close, [4,16,28,40,52])

## Function rolling_window
   
    parameters
    1.series 
    2.window_range: list of integer or range of integers 
    
    return
    rolling_window returns the dataframe which contain column of rolling mean for the window size of each element of list. 

In [6]:
def rolling_window(series, window_range):
  rolling_df=pd.DataFrame()
  for i in  window_range:
    rolling_df['rolling window of size {}'.format(i)]=series.rolling(i).mean()
    
  return rolling_df

#### tcs_rolling_df
    dataframe which contains the column of rolling mean of the window size in range(10,75)

#### infy_rolling_df
    dataframe which contains the column of rolling mean of the window size in range(10,75)


In [7]:
tcs_rolling_df=rolling_window(tcs_data.Close, range(10,75))
infy_rolling_df=rolling_window(infy_data.Close, range(10,75))


## function shocks_data
    the function performs the all required operation which is given below 

     1 Volume shocks - If volume traded is 10% higher/lower than previous day - make a 0/1 boolean time series for shock.
     2 Price shocks - If closing price at T vs T+1 has a difference > 2%, then 0/1 boolean time series for shock.
     3 Pricing black swan - If closing price at T vs T+1 has a difference > 2%, then 0/1 boolean time series for shock.
     4 Pricing shock without volume shock - based on points a & b - Make a 0/1 dummy time series.
     
     
    Parameter
    1.stocks_dataframe
    
    returns
    dataframe with columns names Volume shocks, Price shocks, Pricing black swan, Pricing shock without volume shock

In [8]:
def shocks_data(stock_dataframe):

  shocks_df=pd.DataFrame(index=stock_dataframe.index)
  shocks_df['Volume shocks']=[1 if abs(change) > 0.100 else 0 for change in stock_dataframe.Volume.pct_change()]
  shocks_df['Price shocks']=[1 if abs(change) > 0.020 else 0 for change in stock_dataframe.Close.pct_change()]
  shocks_df['Pricing black swan']=[1 if abs(change) > 0.1000 else 0 for change in stock_dataframe.Close.pct_change()]

  shocks_df['Price shocks without volume shocks']=0
  priceshocks_df=shocks_df[shocks_df['Price shocks']==1]
  ind=priceshocks_df[priceshocks_df['Volume shocks']==0].index
  shocks_df.at[ind, 'Pricing shock without volume shock']=1
  shocks_df['Pricing shock without volume shock'].fillna(0,inplace=True)
  
  return shocks_df
  
  



### tcs_shocks_df
    data frame with the columns Volume shocks, Price shocks, Pricing black swan, Pricing shock without volume shock for tcs stock data 
    
### infy_shocks_df
    data frame with the columns Volume shocks, Price shocks, Pricing black swan, Pricing shock without volume shock for infosys stock data 

In [9]:
tcs_shocks_df=shocks_data(tcs_data)
infy_shocks_df=shocks_data(infy_data)

# Part 2 visualization using bokeh

In [10]:

output_file('plot1.html', mode='inline')


p = figure( x_axis_type='datetime',plot_width=900,plot_height=400, title='close price tcs vs infy')


p.line(tcs_data.index,tcs_data.Close,line_width=2, name='tcs close price', color= 'blue',legend='tcs')
p.line(infy_data.index,infy_data.Close,line_width=2, name='infy close price', color='red', legend='infy')


p.xaxis.formatter = bokeh.models.formatters.DatetimeTickFormatter(
    months=["%Y-%m-%d"])

p.legend.location = "top_right"
p.legend.click_policy="hide"

save(p)

'/home/ravi/plot1.html'

In [11]:
output_file('plot2.html', mode='inline')
p = figure( x_axis_type='datetime',plot_width=900,plot_height=400, title='volume shocks tcs vs infy')


p.line(tcs_shocks_df.index,tcs_shocks_df['Volume shocks'],line_width=2, name='tcs volume shocks', color= 'blue',legend='tcs')
p.line(infy_shocks_df.index,infy_shocks_df['Volume shocks'],line_width=2, name='infy close shocks', color='red', legend='infy')


p.xaxis.formatter = bokeh.models.formatters.DatetimeTickFormatter(
    months=["%Y-%m-%d"])

p.legend.location = "top_right"
p.legend.click_policy="hide"

save(p)

'/home/ravi/plot2.html'

In [12]:
output_file('plot3.html', mode='inline')
p = figure( x_axis_type='datetime',plot_width=900,plot_height=400, title='52 weeks moving average tcs vs infy')


p.line(tcs_moving_av.index,tcs_moving_av['MA of 52 weeks'],line_width=2, name='tcs volume shocks', color= 'blue',legend='tcs')
p.line(infy_moving_av.index,infy_moving_av['MA of 52 weeks'],line_width=2, name='infy close shocks', color='red', legend='infy')


p.xaxis.formatter = bokeh.models.formatters.DatetimeTickFormatter(
    months=["%Y-%m-%d"])

p.legend.location = "top_right"
p.legend.click_policy="hide"

save(p)

'/home/ravi/plot3.html'

In [13]:
output_file('plot4.html', mode='inline')
p = figure( x_axis_type='datetime',plot_width=900,plot_height=400, title='volumeless price movement tcs vs infy')


p.line(tcs_data.index,tcs_data.Close,line_width=2, name='tcs close price', color= 'blue',legend='tcs')
p.circle(tcs_shocks_df[tcs_shocks_df['Pricing shock without volume shock']==1].index,tcs_data.Close.loc[tcs_shocks_df[tcs_shocks_df['Pricing shock without volume shock']==1].index],size=15, color= 'blue',legend='tcs')


p.line(infy_data.index,infy_data.Close,line_width=2, name='infy close price', color='red', legend='infy')
p.circle(infy_shocks_df[infy_shocks_df['Pricing shock without volume shock']==1].index,infy_data.Close.loc[infy_shocks_df[infy_shocks_df['Pricing shock without volume shock']==1].index],size=15, color='red', legend='infy')


p.xaxis.formatter = bokeh.models.formatters.DatetimeTickFormatter(
    months=["%Y-%m-%d"])

p.legend.location = "top_right"
p.legend.click_policy="hide"

save(p)




'/home/ravi/plot4.html'

In [14]:
output_file('plot5.html', mode='inline')

numlines=len(infy_moving_av.columns)
mypalette=Spectral11[0:numlines]
color=itertools.cycle(mypalette)

p = figure(width=900, height=500, x_axis_type="datetime", title='Moving average of tcs') 
for c,col in zip(color,tcs_moving_av.columns):
    p.line(tcs_moving_av.index, tcs_moving_av[col], line_width=5, color=c,legend='{}'.format(col))

p.legend.location = "top_right"
p.legend.click_policy="hide"

save(p)

'/home/ravi/plot5.html'

In [15]:

output_file('plot6.html', mode='inline')

numlines=len(infy_moving_av.columns)
mypalette=Spectral11[0:numlines]
color=itertools.cycle(mypalette)

p = figure(width=900, height=500, x_axis_type="datetime", title='Moving average of tcs') 
for c,col in zip(color,infy_moving_av.columns):
    p.line(infy_moving_av.index, infy_moving_av[col], line_width=5, color=c,legend='{}'.format(col))

p.legend.location = "top_right"
p.legend.click_policy="hide"

save(p)


'/home/ravi/plot6.html'

In [16]:
output_file('plot7.html', mode='inline')
numlines=len(tcs_rolling_df.columns)

p = figure(width=900, height=500, x_axis_type="datetime", title='rolling window up to 75 of tcs') 




number_of_colors = 65

color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

for c,col in zip(color,tcs_rolling_df.columns):
    p.line(tcs_rolling_df.index, tcs_rolling_df[col], line_width=3, color=c)


save(p)


'/home/ravi/plot7.html'

In [17]:
output_file('plot8.html', mode='inline')


p = figure(width=900, height=500, x_axis_type="datetime", title='rolling window up to 75 of tcs') 




number_of_colors = 65

color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

for c,col in zip(color,infy_rolling_df.columns):
    p.line(infy_rolling_df.index, infy_rolling_df[col], line_width=3, color=c)


save(p)


'/home/ravi/plot8.html'

In [18]:
def plot_pacf(stock_dataframe,title_name ,color,save_name):
    
    output_file(save_name, mode='inline')
    
    lags=40
    
    x=range(lags)

    p = figure(width= 400,height=500, title="Partial Autocorrelation PLot {}" .format(title_name))

    partial_autocorr = pacf(stock_dataframe["Close"], nlags=lags)
    p.vbar(x=range(len(partial_autocorr)), top=partial_autocorr, width=0.9,color=color)
    save(p)


In [19]:
plot_pacf(tcs_data, 'TCS', 'blue','plot9.html')

In [20]:
plot_pacf(infy_data, 'INFY','red','plot10.html')